<a href="https://colab.research.google.com/github/lourencocavalcante/Neurocomputacao/blob/main/CAP351_Projeto_01_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/logoinpe.png?raw=true" align="middle">](https://www.gov.br/inpe/pt-br)

[<center><img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/LogoCAP.png?raw=true" width = 500 align="middle"></center>](http://www.inpe.br/posgraduacao/cap/)


---


# **CAP-351 Neurocomputação**

**Professor:**

* Dr. Marcos Goncalves Quiles

**Discente:**
* Lourenço José Cavalcante Neto

# PROPOSTA DE PROJETO 1:


# REDE MLP:: Multi-layer Perceptron (MLP)


1. Selecionar dois datasets (não triviais)

  * Um dataset para classificação
  * Um dataset para regressão
  * Separar em treino/validação/teste

2. Treinar modelos MLP para os dois problemas

3. Considerar:

  * Diferentes topologias (>=5 topologias, variar número de
camadas)

  * Usar o algoritmo original SGD (não usar algoritmos otimizados,
e.g. ADAM)
  
  * Avaliar o impacto do uso do Momentum

  * Avaliar o impacto do uso da regularização (i.e. L2)

4. Ilustrar graficamente a evolução do treinamento (treino/validação).

5. Confeccionar um relatório (reprodutível) contendo os
experimentos e resultados

In [6]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.8.2


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Dataset**

Importando os dados do **disdrômetro RD80 (Joss**) e do radiômetro de microondas **MP3000A**.


In [19]:
#CRIANDO OS DATAFRAMES
#Dados do disdrômetro RD80. Chamado comumente de Joss. 
#Este equipamento mede adistribuição das gotas de chuva que chegam à superfície. (df_rd80_joss)
df_rd80_joss = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/JWD_campina_report_RZLwcDmDtNdrop_b2.1_report.txt',header=None,delim_whitespace=True)


#Dados do radiômetro de microondas MP3000A. Esse instrumento mede diversas variáveis:

#Variáveis de ambiente (dados_ambi_superficie)
df_mp3000A_1 = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/level1_output_sfc.txt',header=None,delim_whitespace=True)
#Radiância no espectro de onda do microonda em diversos canais (dados_med_canais_micro)
df_mp3000A_2 = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/level1_output_bt.txt',header=None,delim_whitespace=True)
#Estimativas de parâmetros de nuvens (dados_est_param_nuvens)
df_mp3000A_3 = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/level2_output_cld_atto.txt',header=None,delim_whitespace=True)

In [20]:
#Os Dataframes estão sem título nas colunas. Vamos adicioná-las.

df_rd80_joss.columns = ['Ano (YYYY)', 'Mes', 'dia', 'H', 'M', 'S','Rain Intensity (mm/h)', 'radar reflectivity (1/mm6m3)','Liquid watercontent (g/m3)',
'Mean weight diameter(mm)', 'Time integration (s)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)']

df_mp3000A_1.columns = ['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S', 'Tamb(K)', 'Rh(%)', 'Pres(mb)', 'Tir(K)', 'Rain', 'DataQuality']
df_mp3000A_2.columns = ['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S', 'Ch 22.234', 'Ch 22.500', 'Ch 23.034', 'Ch 23.834', 'Ch 25.000', 'Ch 26.234', 'Ch 28.000', 'Ch 30.000', 'Ch 51.248', 'Ch 51.760', 'Ch 52.280', 'Ch 52.804', 'Ch 53.336', 'Ch 53.848', 'Ch 54.400', 'Ch 54.940', 'Ch 55.500', 'Ch 56.020', 'Ch 56.660', 'Ch 57.288', 'Ch 57.964', 'Ch 58.800', 'DataQuality']
df_mp3000A_3.columns = ['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S', 'Int. Vapor(cm)','Int. Liquid(mm)','Cloud Base(km)','DataQuality']

In [21]:
#Modificanto o tipo de dado das colunas referentes a Data e Horário para Inteiro
df_rd80_joss[['Ano (YYYY)', 'Mes', 'dia', 'H', 'M', 'S']] = df_rd80_joss[['Ano (YYYY)', 'Mes', 'dia', 'H', 'M', 'S']].astype(int)
df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S']] = df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S']].astype(int)
df_mp3000A_2[['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S']] = df_mp3000A_2[['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S']].astype(int)
df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S']] = df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'H', 'M', 'S']].astype(int)

In [ ]:
for index, row in df_mp3000A_3.iterrows():
  ano = '20' + str(int(row['Ano (YY)']))
  mes = '%02d' % int(row['Mes'])
  dia = '%02d' % int(row['dia'])
  h = '%02d' % int(row['H'])
  m = '%02d' % int(row['M'])
  s = '%02d' % int(row['S'])
    

  #print(mes)
  df_mp3000A_3.loc[index, 'Data'] = str(dia) + '-' + str(mes) + '-'+ str(ano)
  df_mp3000A_3.loc[index, 'Horario'] = str(h) + ':' + str(m) + ':'+ str(s)

In [34]:
df_mp3000A_3

,Mes,dia,Ano (YY),H,M,S,Int. Vapor(cm),Int. Liquid(mm),Cloud Base(km),DataQuality,Data,Horario
0,7,27,21,18,13,45,11.642,19.680,0.00,1.0,27-07-2021,18:13:45
1,7,27,21,18,14,18,11.523,19.684,0.20,1.0,27-07-2021,18:14:18
2,7,27,21,18,14,50,11.544,19.686,0.20,1.0,27-07-2021,18:14:50
3,7,27,21,18,15,23,11.459,19.733,0.00,1.0,27-07-2021,18:15:23
4,7,27,21,18,15,56,11.461,19.722,0.20,1.0,27-07-2021,18:15:56
...,...,...,...,...,...,...,...,...,...,...,...,...
639537,6,1,22,9,17,45,4.498,0.000,0.05,1.0,NaN,NaN
639538,6,1,22,9,18,19,4.161,0.000,0.00,1.0,NaN,NaN
639539,6,1,22,9,18,52,4.299,0.000,0.00,1.0,NaN,NaN
639540,6,1,22,9,19,26,5.400,0.000,0.00,1.0,NaN,NaN
